# Project : Skin Disease Recognition

## DataSet Description: 

The images are meticulously categorized into 22 distinct classes, each corresponding to a specific skin condition:

- Acne
- Actinic Keratosis
- Benign Tumors
- Bullous
- Candidiasis
- Drug Eruption
- Eczema
- Infestations/Bites
- Lichen
- Lupus
- Moles
- Psoriasis
- Rosacea
- Seborrheic Keratoses
- Skin Cancer
- Sun/Sunlight Damage
- Tinea
- Unknown/Normal
- Vascular Tumors
- Vasculitis
- Vitiligo
- Warts




In [14]:
# Import necessary libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array,array_to_img,ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

## Accessing the dataset

In [15]:
# train and test paths
train_dir = "/kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/train"
test_dir = "/kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/valid"

# creating ImageDataGenerator for train and test
train_generator = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range =0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True
    
)
test_generator = ImageDataGenerator(
    rescale = 1.0/255
)

# Accessing the train and test data by performing data Augmentation
train_data = train_generator.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = "categorical",
    seed = 123
)

test_data = test_generator.flow_from_directory(
    test_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = "categorical",
    seed = 123
)

Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


## Creating The VGG16 Model

In [16]:
vgg = VGG16(weights="imagenet",include_top = False, input_shape = (224,224,3))

In [17]:
for layer in vgg.layers:
    layer.trainable = False

x = vgg.output
x  = Flatten()(x)
x = Dense(units = 1000, activation = "relu")(x)
x = Dense(units = 10, activation = "softmax")(x)
model = Model(inputs = vgg.input,outputs= x)

# Compiling the Model
model.compile(
    optimizer = "adam",
    loss = ["categorical_crossentropy"],
    metrics = ["accuracy"]
)

## Creating Callback to save the Best Model

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint  = ModelCheckpoint(
    "best_model.keras",
    monitor = "val_accuracy",
    mode = "max",
    save_best_only = True,
    verbose = 1
)

## Training the Model using train dataset

In [19]:
model.fit(
    train_data,
    validation_data = test_data,
    epochs = 10,
    callbacks = [model_checkpoint],
    verbose = 1
)

Epoch 1/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - accuracy: 0.5500 - loss: 2.4518
Epoch 1: val_accuracy improved from -inf to 0.82028, saving model to best_model.keras
574/574 ━━━━━━━━━━━━━━━━━━━━ 228s 390ms/step - accuracy: 0.5502 - loss: 2.4496 - val_accuracy: 0.8203 - val_loss: 0.5372
Epoch 2/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.7725 - loss: 0.6429
Epoch 2: val_accuracy did not improve from 0.82028
574/574 ━━━━━━━━━━━━━━━━━━━━ 221s 381ms/step - accuracy: 0.7725 - loss: 0.6428 - val_accuracy: 0.8070 - val_loss: 0.5461
Epoch 3/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - accuracy: 0.8112 - loss: 0.5420
Epoch 3: val_accuracy improved from 0.82028 to 0.85649, saving model to best_model.keras
574/574 ━━━━━━━━━━━━━━━━━━━━ 226s 389ms/step - accuracy: 0.8112 - loss: 0.5420 - val_accuracy: 0.8565 - val_loss: 0.4116
Epoch 4/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.8314 - loss: 0.4859
Epoch 4: val_accuracy did not improve from 0.85649
574/574 

In [ ]:
model.summary()

In [30]:
class_names = test_data.class_indices.keys()  # or test_data.classes if you want to get class labels as integers

# Convert to a list if you need it
class_names = list(class_names)
print(class_names)

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


### Preprocessing the data For Prediction

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img

def predict(model,img_path):
    img = load_img(img_path,target_size = (224,224))
    img_array  = img_to_array(img)
    img_array = img_array/255      # Normalizing the image
    img_array = np.expand_dims(img_array,axis=0)
    prediction = model.predict(img_array)
    prediction = np.argmax(prediction)
    prediction = class_names[prediction]
    print(prediction)
    return prediction

In [ ]:
img_path = "/kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/valid/Tomato___Leaf_Mold/146ceb05-4beb-4a1c-9ba5-233f9f1ff0fa___Crnl_L.Mold 6985.JPG"
predict(model,img_path)

## Saving the Final Model Using Tensor Lite

In [ ]:
model.save("model.h5",include_optimizer = False)

In [20]:
# Saving model Using tensorflow lite
from tensorflow.keras.models import load_model
model = load_model("/kaggle/working/best_model.keras")

In [27]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [29]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
lite_model = converter.convert()
with open("lite_model.tflite","wb") as f:
    f.write(lite_model)

Saved artifact at '/tmp/tmp7xwt41jt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132897698391216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698441072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698445296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698439664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698434384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698440896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698435616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698431392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897698434736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897700714576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132897700722